#### 0. Check Information (Debug)

In [ ]:
# Checking Folders Sizes
!du -h -d1

# Check torch version
%pip list -v | grep torch

# Check working directory
!echo "Working directrory:"
%pwd
!du -sh $(pwd)

#### 0.1 Update torch to latest version (if necessary)

In [ ]:
# %pip install --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install --root-user-action=ignore --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 1. Install ComfyUI and dependencies
Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup

from pathlib import Path

OPTIONS = {}

UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}

WORKSPACE = 'ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !echo -= Updating ComfyUI Manager =-
  !git pull
  %cd ../..

!echo -= Install dependencies =-
%pip install torch torchvision torchaudio
%pip install -r requirements.txt
%pip install pyngrok
%pip install tqdm

#### 1.1. Download some models/checkpoints/vae or custom comfyui nodes 
(uncomment the commands for the ones you want)

In [ ]:
model_url = [

    # Download models into folders. Uncomment necessary links.
    # Format: ("URL","PATH","*NAME (optional)"),

    # Checkpoints

### Preview Models
    # ("https://github.com/madebyollin/taesd/raw/main/taesd_decoder.pth", "./models/vae_approx/"),
    # ("https://github.com/madebyollin/taesd/raw/main/taesdxl_decoder.pth", "./models/vae_approx/"),
    # ("https://github.com/madebyollin/taesd/raw/main/taesd_decoder.pth", "./models/vae_approx/"),
    # ("https://github.com/madebyollin/taesd/raw/main/taesdxl_decoder.pth", "./models/vae_approx/"),

### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

    # ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", "./models/checkpoints/"),

# SDXL ReVision
    # ("https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors", "./models/clip_vision/"),

# SD1.5
    # ("https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt", "./models/checkpoints/"),

# SD2
    # ("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors", "./models/checkpoints/"),

# Some SD1.5 anime style
    # ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors", "./models/checkpoints/"),

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
    # ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors", "./models/checkpoints/"),


# unCLIP models
    # ("https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors", "./models/checkpoints/"),
    # ("https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors", "./models/checkpoints/"),


# VAE
    # ("https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.safetensors", "./models/vae/", "sd-vae-ft-mse.safetensors"),
    # ("https://huggingface.co/stabilityai/sd-vae-ft-ema/resolve/main/diffusion_pytorch_model.safetensors", "./models/vae/", "sd-vae-ft-ema.safetensors"),
    # ("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors", "./models/vae/"),
    # ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt", "./models/vae/"),
    # ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt", "./models/vae/"),
# SDXL VAE
    # ("https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors", "./models/vae/"),

# IP-Adapter (downloading to 'ComfyUI_IPAdapter_plus' node folder)
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin", "./custom_nodes/ComfyUI_IPAdapter_plus/models"),
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin", "./custom_nodes/ComfyUI_IPAdapter_plus/models"),
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin", "./custom_nodes/ComfyUI_IPAdapter_plus/models"),
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.bin", "./custom_nodes/ComfyUI_IPAdapter_plus/models"),
# IP-Adapter SDXL
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.bin", ""),
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.bin", ""),
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.bin", ""),
# Image Encoder (for IP-Adapter)
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors", "./models/clip_vision/", "image_encoder_model.safetensors"),
# Image Encoder SDXL (for IP-Adapter)
    # ("https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors", "./models/clip_vision/", "image_encoder_sdxl_model.safetensors"),
       
    
# Loras
    # ("https://civitai.com/api/download/models/10350", "./models/loras/", "theovercomer8sContrastFix_sd21768.safetensors"), #theovercomer8sContrastFix SD2.x 768-v
    # ("https://civitai.com/api/download/models/10638", "./models/loras/", "theovercomer8sContrastFix_sd15.safetensors"), #theovercomer8sContrastFix SD1.x
    # ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/", "./models/loras/", "sd_xl_offset_example-lora_1.0.safetensors"), #SDXL offset noise lora


# T2I-Adapter
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth", "./models/controlnet/"),
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth", "./models/controlnet/"),
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth", "./models/controlnet/"),
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth", "./models/controlnet/"),
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth", "./models/controlnet/"),
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth", "./models/controlnet/"),
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth", "./models/controlnet/"),

# T2I-Adapter SDXL
    # ("https://huggingface.co/TencentARC/t2i-adapter-canny-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", "./models/controlnet/", "t2i-adapter-canny-sdxl-1.0.safetensors"),
    # ("https://huggingface.co/TencentARC/t2i-adapter-sketch-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", "./models/controlnet/", "t2i-adapter-sketch-sdxl-1.0.safetensors"),
    # ("https://huggingface.co/TencentARC/t2i-adapter-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", "./models/controlnet/", "t2i-adapter-openpose-sdxl-1.0.safetensors"),
    # ("https://huggingface.co/TencentARC/t2i-adapter-lineart-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", "./models/controlnet/", "t2i-adapter-lineart-sdxl-1.0.safetensors"),
    # ("https://huggingface.co/TencentARC/t2i-adapter-depth-midas-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", "./models/controlnet/", "t2i-adapter-depth-midas-sdxl-1.0.safetensors"),
    # ("https://huggingface.co/TencentARC/t2i-adapter-depth-zoe-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", "./models/controlnet/", "t2i-adapter-depth-zoe-sdxl-1.0.safetensors"),

# T2I Styles Model
    # ("https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth", "./models/style_models/"),

# CLIPVision model (needed for styles model)
    # ("https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin", "./models/clip_vision/", "clip_vit14.bin"),


# Control-lora stabilityai
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-canny-rank128.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-depth-rank128.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-recolor-rank128.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank128/control-lora-sketch-rank128-metadata.safetensors", "./models/controlnet/"),

# ControlNet
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors", "./models/controlnet/"),

# ControlNet SDXL
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors", "./models/controlnet/"),
    # ("https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors", "./models/controlnet/"),

# GLIGEN
    # ("https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors", "./models/gligen/"),

# ESRGAN upscale model
    # ("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth", "./models/upscale_models/"),
    # ("https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth", "./models/upscale_models/"),
    # ("https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth", "./models/upscale_models/"),
]

# Controlnet Preprocessor nodes by Fannovel16
#%cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py

from tqdm import tqdm
import requests
import re
import os


for _, model in enumerate(model_url):
    model_url = model[0]
    model_path = model[1]
    model_name = model_url.split("/")[-1]
    
    if len(model) > 2:
        model_name = model[2]
    try:
       response = requests.get(model_url, stream=True)
    except:
       print("Unknown error.")

    if not model_name:
        if 'content-disposition' in response.headers:
            d = response.headers['content-disposition']
            model_name = re.findall("filename=(.+)", d)[0].strip("\"';")
    
    file_path = os.path.join(model_path, model_name)
    

    print(f"Downloading now: {model_name} to {model_path}")
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
    with open(file_path, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

    if total_size_in_bytes and progress_bar.n != total_size_in_bytes:
        print(f"ERROR downloading {model_name}")

### 2. Run ComfyUI
Don't forget to fill your [***ngrok token***](https://dashboard.ngrok.com/get-started/your-authtoken) or ***studiolab url*** address
*`example: (https://abcdefghijkl123.studio.us-east-2.sagemaker.aws)`*

In [ ]:
# put your Ngrok token here
ngrok_token=""

# put your SageMaker Studio lab url adress here
sage_url = ""

# Put ComfyUI command line args here
args = "--preview-method auto --enable-cors-header --use-pytorch-cross-attention"


from threading import Timer
from queue import Queue
from pyngrok import ngrok
import os

def ngrok_tunnel(port,queue,auth_token):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port)
    queue.put(url)
    
def launch_comfy(ngrok_token, url_adress, args):
    if ngrok_token:
        ngrok_output_queue = Queue()
        ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
        ngrok_thread.start()
        ngrok_thread.join()
        print(ngrok_output_queue.get())
    elif url_adress:
        print(f'Wait for local URL to appear, then use this link:')
        print(f'{sage_url}/studiolab/default/jupyter/proxy/8188/\n')
    else:
        print(f"-= Please fill your ngrok token or studiolab url address =-\nExiting...")
        return
    os.system("python main.py " + args)
    
launch_comfy(ngrok_token, sage_url, args)